In [1]:
#import sys
#!{sys.executable} -m pip install geopandas
 
import os
import geopandas as gpd
from shapely.geometry import Point
from shapely.prepared import prep
import csv
import subprocess
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.cbook as cbook
from matplotlib.colors import Normalize
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.colors as mcolors
from matplotlib.colors import TwoSlopeNorm
from matplotlib.font_manager import FontProperties

import pandas as pd
import xarray as xr
import numpy as np
import math
from numpy import *
from pylab import *
import pygrib
import pyproj
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.interpolate import griddata
from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager
from datetime import datetime, timedelta
from xarray.backends import NetCDF4DataStore
from netCDF4 import Dataset
import metpy as metpy
import metpy.calc as mpcalc
from metpy.plots import ctables
from metpy.units import units
from metpy.plots import add_metpy_logo, add_timestamp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter
from scipy import interpolate
from scipy.stats import pearsonr
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy import config
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
import glob

In [18]:
datafiles = glob.glob("/scratch/sawyer/wwrf/2017-01-09/wwrf_6km_reanalysis/cf/wrf*")
datafiles.sort()
numfiles=len(datafiles)
print(numfiles)
#print(datafiles)
# Assuming you only want to inspect the first file

72


In [20]:
ncfile = Dataset(datafiles[0])

# Now, you can work with 'datasets', which is a list of xarray.Dataset objects
# For example, to print the contents of the first dataset
print(datasets[0])
lat =ncfile['lat'][:]
wwrflon = ncfile['lon'][:]
nx = lat.shape[0]
ny = lat.shape[1]
print(nx,ny,wwrflon.shape)
## WWRF longitude fix ######
lon =np.where(wwrflon > 0, wwrflon - 360, wwrflon)

storm_total = np.zeros([nx,ny])

for j in range(0,numfiles):
    ncfile = Dataset(datafiles[j])
    precip_bkt = ncfile['precip_bkt'][0]
    storm_total += precip_bkt
    

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy', 'gini']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html

In [ ]:
lcc_projection = ccrs.LambertConformal(
    central_longitude=-125.0,
    central_latitude=40.99998,
    standard_parallels=(50, 32))

In [ ]:
# dark brown for state/coastlines
dark_brown = (0.4, 0.2, 0)
# Create a basic map using Cartopy
fig, ax = plt.subplots(subplot_kw={'projection': lcc_projection})
ax.set_extent([-125,-115,32,42])
ax.add_feature(cfeature.COASTLINE, linewidth=0.5, edgecolor=dark_brown)#Add coastlines)
ax.add_feature(cfeature.STATES, linewidth=0.5,edgecolor=dark_brown)
cp = ax.contourf(lon, lat, storm_total,levels = np.arange(1,20,5),transform=ccrs.PlateCarree()) # here, `precip[0,:,:]` assumes you're plotting the first time step.
fig.colorbar(cp, ax=ax, label="Precipitation (mm)")  # replace 'units' with your actual units

# Add coastlines and gridlines
ax.coastlines()
#ax.gridlines(draw_labels=True)

# Display the plot
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=4, figsize=(24, 16),
                        subplot_kw={'projection': lcc_projection})
axs = axs.ravel()  # Flatten axs

# Loop through the first N-1 elements
for i in range(len(ivt_list)- 1):
    ax = axs[i]
    ax.set_extent([-140, -113, 25, 50])
    ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')
    ax.add_feature(cfeature.STATES, linewidth=1.0, edgecolor='black')
    ax.set_title(titles[i], fontweight='bold')
    cp = ax.contourf(to_np(new_lons), to_np(wrf_lats), to_np(ivt_list[i]), cmap='jet', 
                     levels=wnd_levels_850, transform=ccrs.PlateCarree())
    cp1 = ax.contour(to_np(new_lons), to_np(wrf_lats), to_np(slp_list[i]),
                         colors = 'black',levels=levels_850, linewidths=.7, alpha=1,transform=crs.PlateCarree())
    ax.clabel(cp1, fmt='%1.0f', inline=True,levels=levels_850)


# Handle the last plot separately
ax = axs[len(ivt_list) - 1]
ax.set_extent([-140, -113, 25, 50])
ax.add_feature(cfeature.COASTLINE, linewidth=1.0, edgecolor='black')
ax.add_feature(cfeature.STATES, linewidth=1.0, edgecolor='black')
ax.set_title(titles[-1], fontweight='bold')
cp = ax.contourf(to_np(wwrf_lons), to_np(lat), to_np(ivt_list[-1]), cmap='jet', 
                 levels=wnd_levels_850, transform=ccrs.PlateCarree())
cp2 = ax.contour(to_np(wwrf_lons), to_np(lat), to_np(slp_list[-1]),
                         colors = 'black',levels=levels_850, linewidths=.7, alpha=1,transform=crs.PlateCarree())
ax.clabel(cp2, fmt='%1.0f', inline=True,levels=levels_850)

# Turn off any remaining subplots
for i in range(len(ivt_list), len(axs)):
    axs[i].axis('off')
# Add a single colorbar at the bottom of the figure
cbar_ax = fig.add_axes([0.15, 0.08, 0.7, 0.02])  # Adjust these dimensions to fit the colorbar nicely
cbar = fig.colorbar(cp, cax=cbar_ax, orientation='horizontal')
cbar.set_label("Wind Speed (Kts)", fontsize = 12)
# Adjust layout
fig.tight_layout()
plt.subplots_adjust(bottom=0.15)  # Make space for the colorbar

plt.savefig('850winds_18z_8Jan')
plt.show()